# Automatic Differentiation with Steepest Descent

### Least Squares Steepest Descent

Going to try out a small example of autodiff by using it to solve the steepest descent least squares problem. We then implement a couple tests to ensure our derivatives are correct. We 1) compare to the closed form solution 2) use a test based on Taylor series introduced by Lars. SDLS works as follows. 

\begin{gather}
f(x)=\Vert{Ax-b}\Vert^2 \\
\min_{x}f(x) \\
\end{gather}

We know from vector calculus the steepest descent direction from a point $x$ is the negative gradient evaluated at that point.
So we just need to solve for the solution to the equation. 

\begin{gather}
\nabla f(x)=\frac{\partial}{\partial x} f(x)=A^T(Ax-b)=0
\end{gather}

To solve the exact line search problem to optimize our step size $\alpha$, we solve another optimization. 

\begin{gather}
x_{(i+1)}=x_{(i)}-\alpha\nabla f(x_{(i)}) \\
\min_{\alpha}\Vert{Ax_{(i+1)}-b}\Vert^2 \\
\frac{\partial}{\partial \alpha} f(x_{(i+1)}) = \nabla f(x_{(i+1)})^T \frac{\partial}{\partial \alpha} x_{(i+1)} 
= -\nabla f(x_{(i+1)})^T \nabla f(x_{(i)})
\end{gather}

It's interesting that the step size that guarantees the lowest error in the direction of the previous gradient, is the one 
that makes the next gradient orthogonal to the previous gradient. Hence the zig-zagging path we see in SDLS with exact line search. With further decomposition, we can get a closed form solution to $\alpha$

\begin{gather}
\nabla f(x_{(i+1)})^T \nabla f(x_{(i)})=0 \\
(A^T(Ax_{(i+1)}-b))^T \nabla f(x_{(i)})=0 \\
(A^T(A(x_{(i)}+\alpha\nabla f(x_{(i)})-b))^T \nabla f(x_{(i)})=0 \\
(A^T(Ax_{(i)}+\alpha A\nabla f(x_{(i)})-b))^T \nabla f(x_{(i)})=0 \\
(A^T(Ax_{(i)}-b+\alpha A\nabla f(x_{(i)}))^T \nabla f(x_{(i)})=0 \\
(\nabla f(x_{(i)})+\alpha A^T A\nabla f(x_{(i)}))^T \nabla f(x_{(i)})=0 \\
(\nabla f(x_{(i)})^T+\alpha \nabla f(x_{(i)})^T A^T A) \nabla f(x_{(i)})=0 \\
\alpha = \frac{\nabla f(x_{(i)})^T \nabla f(x_{(i)})}{\nabla f(x_{(i)})^T A^T A \nabla f(x_{(i)})}
\end{gather}

I'm going to use the autodiff on the first optimization problem to compute the gradient of $f(x)$, 
and use this closed form solution on alpha. 

### Quadratic Form Steepest Descent

\begin{gather}
f(x)=\frac{1}{2}x^TAx-b^Tx+c \\
\min_{x}f(x) \\
\nabla f(x)=\frac{\partial}{\partial x} f(x)=Ax-b=0 \\\\
x_{(i+1)}=x_{(i)}-\alpha\nabla f(x_{(i)}) \\\\
\min_{\alpha} \frac{1}{2} x_{(i+1)}^TAx_{(i+1)} -b^Tx_{(i+1)}+c\\
\frac{\partial}{\partial \alpha} f(x_{(i+1)}) = \nabla f(x_{(i+1)})^T \frac{\partial}{\partial \alpha} x_{(i+1)} 
= -\nabla f(x_{(i+1)})^T \nabla f(x_{(i)}) \\\\
\nabla f(x_{(i+1)})^T \nabla f(x_{(i)})=0 \\
(A(x_{(i)}+\alpha\nabla f(x_{(i)})-b)^T \nabla f(x_{(i)})=0 \\
\alpha = \frac{\nabla f(x_{(i)})^T\nabla f(x_{(i)})}{\nabla f(x_{(i)})^T A\nabla f(x_{(i)})}
\end{gather}

### Code for Least Squares

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import autograd.numpy as np
import autograd as ad

In [21]:
def loss(x, A, b):
    """
    Computes least-squares loss function for a linear system. 
    """
    return .5 * np.linalg.norm(A @ x - b)**2

def explicit_loss_grad(x, A, b):
    """
    Explicit calculation of gradient of least squares loss function
    """
    return A.T @ (A @ x - b)

def line_search(alpha, x, A, b, d):
    """
    Computes least squares loss function wrt to alpha
    """
    return .5 * np.linalg.norm(A @ (x + alpha * d) - b)**2

loss_grad = ad.grad(loss)
line_search_grad = ad.grad(line_search)

def SDLS(A, x, b, maxIter):
    n = A.shape[1]
    his = np.zeros(maxIter)
    grad = np.zeros((n, maxIter))
    Xall = np.zeros((n, maxIter))
    for i in np.arange(maxIter):
        his[i] = np.linalg.norm(A @ x - b)**2
        # ************
        d = -loss_grad(x, A, b)
        #*************
        grad[:,i] = d[:,0]
        Ad = A @ d
        alpha = (d.T @ d) / (Ad.T @ Ad)
        x = x + alpha * d
        Xall[:,i] = x[:,0]
    return x, his, grad, Xall
                               

In [22]:
A = np.array([[3, 2], [2, 6]])
b = np.array([[2],[-8]])
x = np.zeros_like(b)

In [38]:
sol, his, grad, Xall = SDLS(A, x, b, 70)

In [39]:
Xall

array([[-0.2152675 ,  1.28907243,  1.21255263,  1.747291  ,  1.72009098,
         1.9101711 ,  1.90050248,  1.96806908,  1.96463224,  1.98864971,
         1.98742804,  1.99596538,  1.99553112,  1.99856584,  1.99841148,
         1.99949021,  1.99943534,  1.99981879,  1.99979928,  1.99993559,
         1.99992865,  1.9999771 ,  1.99997464,  1.99999186,  1.99999098,
         1.99999711,  1.9999968 ,  1.99999897,  1.99999886,  1.99999963,
         1.9999996 ,  1.99999987,  1.99999986,  1.99999995,  1.99999995,
         1.99999998,  1.99999998,  1.99999999,  1.99999999,  2.        ,
         2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
         2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
         2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
         2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
         2.        ,  2.        ,  2.        ,  2.        ,  2.        ,
         2.        ,  2.        ,  2.        ,     

In [40]:
his

array([6.80000000e+01, 2.41715373e+01, 8.59210613e+00, 3.05418256e+00,
       1.08565129e+00, 3.85909716e-01, 1.37176928e-01, 4.87614299e-02,
       1.73329224e-02, 6.16122618e-03, 2.19009277e-03, 7.78498665e-04,
       2.76728081e-04, 9.83668110e-05, 3.49658389e-05, 1.24290894e-05,
       4.41809115e-06, 1.57047140e-06, 5.58245707e-07, 1.98436131e-07,
       7.05368582e-08, 2.50732985e-08, 8.91264958e-09, 3.16812415e-09,
       1.12615340e-09, 4.00306748e-10, 1.42294552e-10, 5.05805598e-11,
       1.79795573e-11, 6.39108147e-12, 2.27179802e-12, 8.07541923e-13,
       2.87051910e-13, 1.02036558e-13, 3.62703027e-14, 1.28927797e-14,
       4.58291689e-15, 1.62906110e-15, 5.79072511e-16, 2.05839361e-16,
       7.31685645e-17, 2.60088240e-17, 9.24515442e-18, 3.28630891e-18,
       1.16813547e-18, 4.15220896e-19, 1.47595592e-19, 5.24638974e-20,
       1.86509335e-20, 6.63032453e-21, 2.35745779e-21, 8.38204444e-22,
       2.97759314e-22, 1.05794745e-22, 3.75512598e-23, 1.33424849e-23,
      

In [41]:
grad

array([[-1.00000000e+01,  9.84766335e+00, -3.55463784e+00,
         3.50048768e+00, -1.26354502e+00,  1.24429660e+00,
        -4.49144494e-01,  4.42302377e-01, -1.59654601e-01,
         1.57222477e-01, -5.67514288e-02,  5.58868965e-02,
        -2.01730776e-02,  1.98657677e-02, -7.17079851e-03,
         7.06156097e-03, -2.54895917e-03,  2.51012918e-03,
        -9.06062674e-04,  8.92260019e-04, -3.22072467e-04,
         3.17166123e-04, -1.14485098e-04,  1.12741070e-04,
        -4.06953061e-05,  4.00753674e-05, -1.44657075e-05,
         1.42453418e-05, -5.14203513e-06,  5.06370311e-06,
        -1.82780726e-06,  1.79996308e-06, -6.49719293e-07,
         6.39821692e-07, -2.30951673e-07,  2.27433452e-07,
        -8.20949571e-08,  8.08443663e-08, -2.91817814e-08,
         2.87372584e-08, -1.03730615e-08,  1.02150688e-08,
        -3.68726427e-09,  3.63106789e-09, -1.31069999e-09,
         1.29068489e-09, -4.65897543e-10,  4.58790783e-10,
        -1.65595981e-10,  1.63101532e-10, -5.88631366e-1

Why does it return nan at 100 iterations? NaN's start appearing in the loss when the loss hits zero, but one before that in the X's and the gradient. Can see this by seeing how back propagation/reverse accumulation works. 

\begin{gather}
y = D(C(x)) \\ 
\frac{\partial y}{\partial x} = \frac{\partial d}{\partial c} \frac{\partial c}{\partial x} \\ 
\end{gather}